### A k-mer tends to have a higher probability when it is more similar to the consensus string of a profile.

### Greedy motif search:

#### Given a profile matrix Profile, we can evaluate the probability of every k-mer in a string Text and find a Profile-most probable 

#### k-mer in Text, i.e., a k-mer that was most likely to have been generated by Profile among all k-mers in Text. 

#### For example, ACGGGGATTACC is the Profile-most probable 12-mer in GGTACGGGGATTACCT. Indeed, every other 12-mer in 

#### this string has probability 0. In general, if there are multiple Profile-most probable k-mers in Text, then we 

#### select the first such k-mer occurring in Text.

## Profile-most Probable k-mer Problem: Find a Profile-most probable k-mer in a string.

    Input: A string Text, an integer k, and a 4 × k matrix Profile.
    Output: A Profile-most probable k-mer in Text.


In [37]:
import sys
import math
import numpy as np

In [38]:
def Profile(motifs):
    k = len(motifs[0])
    n = len(motifs)
    s = 1 / n
    seq1 = 'ACGTacgt01230123'
    seq_dict = { seq1[i]:int(seq1[i+8]) for i in range(8) }
    P = [[0 for _ in range(k)] for __ in range(4)]
    for motif in motifs:
        for i in range(k):
            P[seq_dict[motif[i]]][i] += s
    return P

In [39]:
def Pr(pattern, profile):
    seq1 = 'ACGTacgt01230123'
    seq_dict = { seq1[i]:int(seq1[i+8]) for i in range(8) }
    p = 1
    k = len(pattern)
    for i in range(k):
        p *= profile[seq_dict[pattern[i]]][i]
    return p

In [40]:
def ProfileMostPr_kmer(seq, k, profile):
    l = len(seq)
    pmax = -1
    imax = -1
    for i in range(l-k+1):
        p = Pr(seq[i:i+k], profile)
        if p > pmax:
            pmax = p
            imax = i
    return seq[imax:imax+k]

In [41]:
def Score(motifs):
    k = len(motifs[0])
    n = len(motifs)
    seq1 = 'ACGTacgt01230123'
    seq_dict = { seq1[i]:int(seq1[i+8]) for i in range(8) }
    P = [[0 for _ in range(4)] for __ in range(k)]
    for motif in motifs:
        for i in range(k):
            P[i][seq_dict[motif[i]]] += 1
    Sm = 0
    for i in range(k):
        Sm += max(P[i])
    return n * k - Sm

In [42]:
def PseudoProfile(motifs):
    k = len(motifs[0])
    n = len(motifs)
    s = 1 / (n + 4)
    seq1 = 'ACGTacgt01230123'
    seq_dict = { seq1[i]:int(seq1[i+8]) for i in range(8) }
    P = [[1 for _ in range(k)] for __ in range(4)]
    for motif in motifs:
        for i in range(k):
            P[seq_dict[motif[i]]][i] += s
    return P

In [43]:
def Count(motifs):
    k = len(motifs[0])
    n = len(motifs)
    seq1 = 'ACGTacgt01230123'
    seq_dict = { seq1[i]:int(seq1[i+8]) for i in range(8) }
    P = [[0 for _ in range(k)] for __ in range(4)]
    for motif in motifs:
        for i in range(k):
            P[seq_dict[motif[i]]][i] += 1
    return P

In [44]:
def GreedyMotifSearch(dna, k, t):
    BestMotifs = [dna[i][0:k] for i in range(t)]
    BestScore = float('inf')
    dna1 = dna[0]
    l1 = len(dna1)
    for i in range(l1-k+1):
        motifs = []
        motifs.append(dna1[i:i+k])
        for i in range(1, t):
            P = Profile(motifs)
            motifs.append(ProfileMostPr_kmer(dna[i], k, P))
        currScore = Score(motifs)
        if currScore < BestScore:
            BestMotifs = motifs
            BestScore = currScore
    return BestMotifs

In [45]:
def GreedyMotifSearch2(dna, k, t):
    #GreedyMotifSearch with pseudocounts
    BestMotifs = [dna[i][0:k] for i in range(t)]
    BestScore = float('inf')
    dna1 = dna[0]
    l1 = len(dna1)
    for i in range(l1-k+1):
        motifs = []
        motifs.append(dna1[i:i+k])
        for i in range(1, t):
            P = PseudoProfile(motifs)
            motifs.append(ProfileMostPr_kmer(dna[i], k, P))
        currScore = Score(motifs)
        if currScore < BestScore:
            BestMotifs = motifs
            BestScore = currScore
    return BestMotifs 

In [49]:
dna="TCTGCAGATGGCGCGCAAAAAAAAGGCTGTAATTGGATGAATGTAACGTCTGAATTGAATGTCCGGAGTTGTCAGTCTAGGCGCGGGGCAAGCAGCAGTGACATACTAACTATCCAAACCAAGTAAGATGTTTTCTAAAGGGTCCACGAAAATGTG TCAATCACATACAGTAATGATGCGAAAATCGGTTCGAAGAATGATGTTTTGGACTTTTGACCACACGACAGGCAACGGGGTAATTGGCTATATTGAGTCAGGCTCGCGAGCGCCATTCGGGATCGAGCATGCAAAAACAAATTTGGTCACTTCCTG TGACAATCGGAGCACGCCAGTTACTCCTCTGGGCTAGGTGGTGTCTTGTCTACCTTGCCATACGCACAAAGCATTCAAATATGCCCAAGTGTACCTAAGTAGGTTGTAACTCGCGTGAGTATACCTCAGTATCTAGCTTAGTCATACGACTCTGTG CGGAGTATTCTCAAATCGGGCTGCAAGAACGGTGTCTAGCTCTAGACCGGGCCTATTTACAATCTAATCCGACCCTATGGCTATTCTACCTTTATATAGTATAGACTCTTCACGACGCTAGTCACGTGCACCAGTGTCACTGACGGAAGCCTGCCC ACAACCGGATGAACGGCATATATGTTCTATCGACAACGAGGGGGCCACATCGGGCTTCTAGGTATCTTTAAAAATCTTCCTATTATCCATAGCTTAACAATGTCGGGTTAGGAATATCATAAGCAGGATGTTCTTCGTTGTAGGTAATTAGACCAC GACCTGTATAAGTTGACCGATCTAAACTCAAATGTTACCGTTCCCGCTCCATAAAAATGGAGCCCGCCCGCATTAAGGAGAAGGTTTCAGCTTGGGTGGATTAAGAACTGAAATACTAAGAAGTAAGTTGTCTGATTCGTCAAGCTCACAGACTCC AAGAAGGCTGTGTCAATGAAGTTGTGGGCGCCTTACTGATCCTGCACAAGACGCTTCAGGGAGAGAAAATCGGATTTTTGAGAATGCGGTTAGCAGATCCGCCCATAGCTTGATGCTAATGAGCAACACATTGGAGCCTCTACTCCTTTTAATTTT CTGATTCGAAAGCAGTGCACGCCCTTTACAATAGAGGTAAAACGTTCTAGATTAGACGTTCAAAACTCTGCACTCACCATCGTGTGAAGAAATCGTAGATGGCCAAGCACGGCACCGGTCAAGGAAGGTGTAGGCGACCTGATGATTTTCCGCGGT AGCATCCGAATGTTCAGTCAATCCGCTCACCTCCATAGTTCGTCATTGAAGTAAGATGTTCAACTACCTGTTAGCGATTCTACATACGCTGAAGCTACCCAACTCGCAGGATGATATTGTTACCTTACAATACAGACACCTGCGATCCCTAGTTAG GGCCTCCGCCGGTACCCGTCCTCCGCCATGGTTAAGAAGAACGTTGTGGATTCGTGGGAGACACTCTTTGGGGACACACAGATCGCGTAGGTTGGATATGACCAGCACCCTCTAACTCCGGAGGACGGTCAATATCTCCTAAGAGGCTAACTATCG CCGCGCGTTCCGGTAGTCTATGGAAAGAGATTGCCCAATCTGTTGGTTAACTTCTAGAATAGAACACCCAGTGCCATCCTGATGGGTCAGTAGTTTGACCGCAGGTTTAAGGAGGTTGTGTTCGAGAGGAAATTTCCGGCCCAGGCCGCTGTTCCC GTTGTGTTTCCTGCATCATTGTACAAGTAAGATGTACCGGTACCATCCGAGATATGTACAATTGGACGTGGTAATCTCTAATACCTAAGCAGCTACAGTTACCCATTTCAGATCTCAAGCTGGCGGCGCAGCATTTAACGACGAGAAATTTAAGTA ACGTGTCCGAGGATCAAGTGACCCCGAGAGCTTCCTAGAGGATGATACACCTCCACGGGCACGTGGACTCCCGTATTTCCTCAGGCATCTTCTAGGGCTAACTACCACAAGAAGGGTGTCCTGCTGACATCACGTTACCCATCTTCGGGCACTTAG GTACCCCAGGGCCAGACCCCTATTAAGTATGTTGTCGTCCACTTGGGATGCTGTGGTGCGTAAAACTAGTACAGAATACCTTGTTGGAAGTTCTGCTCAAGAGCCTGTTTAAGGCGCATACTGTGCCGCCTTCCGCTGTTGCCCATTTTACACGTG GTGAAAGATTCTATATCCTCGATGTTTTTATGTGTGCCATGAGTTTAGATGTCACGTGACTTTGAAAACGCATTGATAGGGGTCCACTCTCACATATAGTCGGGTTTATGTTGGTTAGTCAAGGAGGATGTTGCGATGGCGCGTGAAGCTATTCGC GATGTATGCGCTTGTGGCGTCCACAAGCAGGGTGTTTCGCGAGGAAAAGATTCAATTGCAACCAAGCCACATGTAGAGGCACCCGACAACAGGTTGGGGAATCTCAATTTGCGACCCAGCGCCAGAGGGCCATTGAGACCTGGTATGACAGATGCA CCACACCCGAGCATACAGCTTGACACTGAGAATCTAGACCAAGGACGACTCTATCAACCTTTCTTTATGCAGAAGGTTCATGTGGTACCTCATGACAAGAAAGTTGTTGCGATTAGATGAGTCGTATATCAAGCGCTGAACATCCATCGTCCCACT ACCACTCCAGCCCACCAGATACTTTTGGCGTACACGAGCTGCTCGTTGCACATACTCATGTTGAGTTAATGAAAATTATGATCGATTAGATTTTAGGACGTACAGGCTCAAGGTTCTGCAAAGTAGGTTGTACGTGGTAGCGACATGTGCGGAGGC GAGTCGATTCTGAGAACCGACCACCCAGGCGATCCGGGGAGGGTCTATCTGGCGGTACCCGCGACGTTAAAAAAGAAAGATGTAGACGACAGGCGGACCCTGGATTCAAGTGGTATCCATGCAAAGAATCACAGTTACGTCATTATACTAACGTCA GGCGTCACATAATCTTCAGAAGATGTCGCATGGCGCAAGCATGGTGTGCACGGCGCGAGCTGTATACTGTTCCATAACGTGGAGCTAGCCTTGGTCATAGGTGAGATCGTGACGGGACTCGCCGAGGCGAGAGTTGGGGCACGAGTTGGTATGACC AGGGGTGGCACCAAATCCTGAGGTCCCAATGTCCGACCGTTCCGTGGTCGTCTTATAACTCGCATCTAGAATAAGGATGGTGTATAACTGATTTCGCCACGAGATGTAATTAAGTTCCTCATCAGTTAGGGATCAGGGCAGAGTCATGGTTTGCAA GGGCGAGCCTTGATCACTAGCGTCAGTTGGTCAAATATTTTATTCGACCTAATGCATTCACCATTAAGTAAGTTGCGGTGATTACATCAATCCACTCTGAAAGCAGCTACTGTCCAGTAGAAGAAGGGTGTGACACCTCAAATTATAACCACGTAA CGGGGCTTTCCAGATTCATAATAGAGTAACACGATCATGAGATTTCCTAACGAGTACACCGAGCTCCGTCCTCACTCATGCACTACCTGGAAAACCATGGTGGACGGCAAGCAAGATGTGAAGGTCAACGCCCCCTAGCGACCGGGGAATTTCAAG AAGAAAGTTGTGTACGAAACGTAAGTCTGTAAAAGAACTCAAGCTCTCGATTCAGAATATAGGCAATCAGTTGGAGTCTATCAACGTGCGTATATGTGCGCTCTCATTGAGTGGCTCTTAGATAGTATGATGAAGAACGTCGGCCGGTACCTTTCA GATACCAGAGGAAGCCAGGCTTTGGTAGGGCGTATTTCTTGGAGTTCAGACCACTCGACACCGTGCGTTTGAGCTTTTTCTGCATGAAGCATCCCAACTGGCTATTGCAAGTAAGGTGTCGTGACTTATTAGGCTCGCCTACGAAAACTGATCCAA"

In [50]:
BestMotifs = GreedyMotifSearch2(dna, 12,25)

In [51]:
for motif in BestMotifs:
    print(motif)

T
C
T
G
C
A
G
A
T
G
G
C
G
C
G
C
A
A
A
A
A
A
A
A
G


In [52]:
import math as mt


In [56]:
A = 0.2*mt.log2(0.2)+0.3*mt.log2(0.3)+0.5*mt.log2(0.5)+0.9*mt.log(0.9)+0.4*mt.log(0.4) 

In [57]:
A

-1.94681605406904